In [120]:
#Useful libraries
import pandas as pd

In [121]:
#Importing datasets
#Books
books_file = 'datasets/books_rs/books.csv'
df_books = pd.read_csv(books_file)

#Movies
movies_file = 'datasets/books_rs/movies.csv'
df_movies = pd.read_csv(movies_file)

Let's make the two datasets omogeneus.  First reordering.
Now we separate the ratings from the actual content, before removing the unnecessary attributes.

In [122]:
#Reindexing columns
df_books = df_books.reindex(columns=['title', 'categories', 'authors', 'description', 'ratings_count', 'average_rating', 'published_year', 'subtitle', 'isbn13'])
df_movies = df_movies.reindex(columns=['Series_Title', 'Genre', 'Director', 'Overview', 'No_of_Votes', 'IMDB_Rating', 'Released_Year'])

#Drop unused columns
df_books = df_books.drop(['published_year', 'subtitle', 'isbn13'], axis=1, errors='ignore')
df_movies = df_movies.drop(['Released_Year'], axis=1, errors='ignore')

#Rename the columns to be the same
df_movies.columns = ['title', 'categories', 'authors', 'description', 'ratings_count', 'average_rating']

#Add a column to identify the type of content
df_books['content_type'] = 'book' 
df_movies['content_type'] = 'movie'

#Save separated ratings and content dataframes for books and movies
df_books_ratings = df_books[['title', 'ratings_count', 'average_rating']]
df_books_content = df_books.drop(['ratings_count', 'average_rating'], axis=1, errors='ignore')

df_movies_ratings = df_movies[['title', 'ratings_count', 'average_rating']]
df_movies_content = df_movies.drop(['ratings_count', 'average_rating'], axis=1, errors='ignore')

#Normalize the books ratings
df_books_ratings.loc[:, 'average_rating'] = df_books_ratings['average_rating'] * 2



Now we can merge the two content dataframes, keeping an eye on the dimensions.

In [123]:
print("df_books_content dimensions:", df_books_content.shape)
print("df_movies_content dimensions:", df_movies_content.shape)
cross_content = pd.concat([df_books_content, df_movies_content])
print("cross_content dimensions:", cross_content.shape)



df_books_content dimensions: (6810, 5)
df_movies_content dimensions: (1000, 5)
cross_content dimensions: (7810, 5)


Now we condense all the non title attributes in one TAGS column.

In [124]:
cross_content['tags'] = cross_content.apply(lambda row: '; '.join([str(row[col]) for col in cross_content.columns if col != 'title']), axis=1)
cross_content = cross_content.drop(['categories', 'authors', 'description', 'content_type'], axis=1, errors='ignore')

,title,tags
0,Gilead,Fiction; Marilynne Robinson; A NOVEL THAT READ...
1,Spider's Web,Detective and mystery stories; Charles Osborne...
2,The One Tree,American fiction; Stephen R. Donaldson; Volume...
3,Rage of angels,"Fiction; Sidney Sheldon; A memorable, mesmeriz..."
4,The Four Loves,Christian life; Clive Staples Lewis; Lewis' wo...


The final step saving fuck all.

In [125]:
cross_content.to_csv('cross_content.csv', index=False)

Now if the datasets are omogeneus we can merge and apply preprocessing. 